# Omnidirectional Transfer for Quasilinear Lifelong Learning

[![Paper](https://img.shields.io/badge/Paper-arXiv-green)](https://arxiv.org/pdf/2004.12908.pdf)

## Introduction

* In biological learning, the learning is lifelong, with agents conitnually building on past knowledge and experiences, improving on many tasks given data associated with any task. (e.g. learning a second language improves the individual's performance in his native language)
* Even though classical ML can simultaneously optimize for multiple tasks, if is difficult to sequentially optimize for multiple tasks.
* Catastrophic forgetting: performance on the prior tasks drop precipitously upon training on new tasks.
* Biological learning doesn't suffer from catastrophic forgetting
* Two camps of overcoming catastrophic forgetting:
    * Fixed resources, and therefore, reallocate resources (compressing representations) to incorporate new knowledge (**biologically, this is adulthood**)
    * Adds resources to incorporate new knowledge (**biologically, this is (juvenile) development**)
* The inability to omnidirectionally transfer is one of the key liming factors of AI.
* In ProgNN, new tasks yield additional representational capacity. ProgNN can transfer forward, but they cannot transfer backward.

## Main Contributions

* Representational ensembling that enables omnidirectional transfer via an "Omni-voter" layer
* Computational time reduced from quadratic to quasilinear
* Two types of omnidirectional learning algorithms: 
    * Omnidirectional Forests (ODIF)
    * Omnidirectional Networks (ODIN)
* ODIN and ODIF are resource building (juvenile) but since they can leverage prior representations, they can convert in to an resouce recruiting (adult) state too. 

## Background

### Classical ML

* Consider RVs $(X, Y) \sim P_{X,Y} where $X \sim \mathcal{X}$ is the input and $Y \sim \mathcal{Y}$ is the label. 
* $P_{X,Y} \in \mathcal{P}_{X,Y}$ is the joint distribution of $(X, Y)$
* Let $l \colon \mathcal{Y} \times \mathcal{Y} \longrightarrow (0, \infty]$ be a loss function
* The goal of classifcal ML is to find the hypothesis (predictor/ decision rule) $h \colon \mathcal{X} \longrightarrow \mathcal{Y}$ that minimizes te expected loss or *risk*,
    $$ R(h) = \mathbb{E}_{X, Y}[l(h(X), Y)] $$
* A learning algorithm is a function $f$ that maps a dataset $\bf{S}_n = \{ X_i, Y_i \}_{i=1}^n$. 
* If $n$ samples of $(X, Y)$ is i.i.d from some true but unknown $P_{X,Y}$, the generalization error or expected risk is given by,
    $$ \mathbb{E}[R(f(\bf{S}_n))]$$
* The goal: choose a learner $f$ that learns a hypothesis $h$ that has a small generalization error for the given task.

### Lifelong Learning (LL)

* Lifelong learning generalizes classifcal ML in the following ways:
    * environment of $\mathcal{T}$ tasks instead of a single task
    * data arrive sequentially, instead of batch mode
    * computational complexity contraints on the learning algoritm and hypotheses
* Goal of LL: given new data and a new task, use all the exisiting data to achieve a lower generalization error on the new task, while also using the new data to obtain a lower generalization error on the previous tasks.
* previous work: 
    * updating a fixed parametric model as new tasks arrive
    * adding resources as new tasks arrive
    * store/ replay previously encountered data to reduce forgetting
* Task-aware: the learner is aware of all-task details for all tasks $h \colon \mathcal{X} \times \mathcal{T} \longrightarrow \mathcal{Y}$.
* Task-unaware (task-agnostic): learner may not know that the task has changed at all $h \colon \mathcal{X} \longrightarrow \mathcal{Y}$.

### Reference Algorithms

**Resource Building Algorithms**: Progressive Neural Nets (ProgNN), Deconvolution-Factorized CNNs (DF-CNNs)

**Fixed Capacity Algorithms**: Elastic Weight Consolidation (EWC), Online-EWC (O-EWC), Synaptic Intelligence (SI), Learning without Forgetting (LwF), ‘None’ and two variants of exact replay (Total Replay and Partial Replay).

## Evaluation Criteria





===============================

* The concept of omnidirectional transfer of knowledge is proposed to overcome the issue of catastrophic forgetting. 

* Through omnidirectional transfer, it is possible to realize the goal of lifelong learning, which is to improve the performance on a new task using knowledge about existing tasks and their data, while improving the performance on the previous tasks using the knowledge about new tasks and their data. 

* Introducing the concept of representational ensembling (through the Omni-voter) to accomplish the omnidirectional transfer is quite intuitive and its success is backed by experimental evidence mentioned in the manuscript.

* This work further uses progressive learning concepts to incorporate resource building and resource recruitment into the proposed algorithms. 

# Potential Future directions? 

* How can we expand the proposed learning framework into task-agnostic situations? 
* How can we integrate tasks from different domains? (for instance, training an algorithm to learn two distinctive computer vision tasks. e.g. task 1: object recognition and task 2: semantic segmentation) 
* Would there be an upper bound / practical limitation for increasing representational capacity as the number of tasks fed into the network increases?
* How can we extend the concept of omnidirectional transfer to learning algorithms other than PLNs and PLFs? (and for the situations where the hypotheses that cannot be readily decomposed into discernible transformers, voters, and deciders)

